In [ ]:
import os
os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

Current working directory is c:\Users\blond\Documents\NssProjects\accre-green-bean-casserole


In [2]:
import pandas as pd
from scripts.get import log_to_df, df_to_datelist
from collections import Counter

In [3]:
jobs = pd.read_csv("data/fullsample.csv")


In [4]:
jobs['BEGIN'] = pd.to_datetime(
    jobs['BEGIN'], 
    errors='coerce'
)
jobs['END'] = pd.to_datetime(
    jobs['END'], 
    errors='coerce'
)

C:\Users\blond\AppData\Local\Temp\ipykernel_1048\1972129250.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  jobs['END'] = pd.to_datetime(


In [ ]:
# had to drop the na to get the drop to work
jobs = jobs.dropna(subset = 'END')

In [6]:
# assigning min and max timestamps to remove from the jobs dataframe.
min_timestamp = pd.to_datetime('2020-10-16 10:37:44.163454')
max_timestamp = pd.to_datetime('2021-10-07 20:41:11')

In [7]:
jobs = jobs.loc[jobs['END'] >=  min_timestamp]
jobs = jobs.loc[jobs['END'] <= max_timestamp]

In [8]:
jobs['END'].min()

Timestamp('2020-10-16 10:37:45')

In [9]:
jobs['END'].max()

Timestamp('2021-10-07 20:41:11')

In [10]:
ce5 = log_to_df('data/slurm_wrapper_ce5.log')
ce6 = log_to_df('data/slurm_wrapper_ce6.log')
error_list = df_to_datelist(ce6) + df_to_datelist(ce5)

In [14]:
len(error_list)

3296

In [81]:
# changing way doing the column for day/hour, then added ones for every 15, 30, 45 mins this is not doing what I wanted, I wanted rounded numbers, and this was not in the right intervals
# jobs['END'].dt.to_period('H')
# jobs['END'].dt.to_period('15T')
# jobs['END'].dt.to_period('30T')
# jobs['END'].dt.to_period('45T')
# df['date'].dt.to_period('H')


# this worked better but would need to change for each interval, and then also figure out how to match this to the errors list
# Group by 15-minute intervals
jobs_15min = jobs.set_index('END').groupby(pd.Grouper(freq='15min')).size().reset_index(name = 'JobEndCount')
jobs_30min = jobs.set_index('END').groupby(pd.Grouper(freq='30min')).size().reset_index(name = 'JobEndCount')
jobs_45min = jobs.set_index('END').groupby(pd.Grouper(freq='45min')).size().reset_index(name = 'JobEndCount')
jobs_60min = jobs.set_index('END').groupby(pd.Grouper(freq='60min')).size().reset_index(name = 'JobEndCount')




In [85]:
jobs_15min = jobs_15min.rename(columns={'END': 'end_group'})
jobs_30min = jobs_30min.rename(columns={'END': 'end_group'})
jobs_45min = jobs_45min.rename(columns={'END': 'end_group'})
jobs_60min = jobs_60min.rename(columns={'END': 'end_group'})

In [86]:
jobs_60min.head()

,end_group,JobEndCount
0,2020-10-16 10:00:00,181
1,2020-10-16 11:00:00,1064
2,2020-10-16 12:00:00,824
3,2020-10-16 13:00:00,605
4,2020-10-16 14:00:00,1071


In [72]:
# made this into a dataframe, going to group like the jobs and count how many errors are in each time grouping 
error_df = pd.DataFrame({'error_timestamp': error_list})

In [78]:
errors_15min = error_df.set_index('error_timestamp').groupby(pd.Grouper(freq='15min')).size().reset_index(name = 'ErrorCount')
errors_30min = error_df.set_index('error_timestamp').groupby(pd.Grouper(freq='30min')).size().reset_index(name = 'ErrorCount')
errors_45min = error_df.set_index('error_timestamp').groupby(pd.Grouper(freq='45min')).size().reset_index(name = 'ErrorCount')
errors_60min = error_df.set_index('error_timestamp').groupby(pd.Grouper(freq='60min')).size().reset_index(name = 'ErrorCount')


In [79]:
errors_15min = errors_15min.rename(columns={'END': 'end_group'})
errors_30min = errors_30min.rename(columns={'END': 'end_group'})
errors_45min = errors_45min.rename(columns={'END': 'end_group'})
errors_60min = errors_60min.rename(columns={'END': 'end_group'})

In [82]:
errors_15min.head()

,error_timestamp,ErrorCount
0,2020-10-18 06:15:00,1
1,2020-10-18 06:30:00,1
2,2020-10-18 06:45:00,2
3,2020-10-18 07:00:00,0
4,2020-10-18 07:15:00,0


In [95]:
job_errors_60min = (
    pd.merge(
        left = jobs_60min,    
        right = errors_60min,
        left_on = 'end_group',
        right_on = 'error_timestamp',
        how = 'left')
    .drop(columns = 'error_timestamp')
    )

In [97]:
job_errors_45min = (
    pd.merge(
        left = jobs_45min,    
        right = errors_45min,
        left_on = 'end_group',
        right_on = 'error_timestamp',
        how = 'left')
    .drop(columns = 'error_timestamp')
    )

In [98]:
job_errors_30min = (
    pd.merge(
        left = jobs_30min,    
        right = errors_30min,
        left_on = 'end_group',
        right_on = 'error_timestamp',
        how = 'left')
    .drop(columns = 'error_timestamp')
    )

In [99]:
job_errors_15min = (
    pd.merge(
        left = jobs_15min,    
        right = errors_15min,
        left_on = 'end_group',
        right_on = 'error_timestamp',
        how = 'left')
    .drop(columns = 'error_timestamp')
    )

In [103]:
job_errors_15min['ErrorCount'].value_counts()

ErrorCount
0.0     31853
1.0      1424
2.0       348
3.0       176
4.0        62
5.0        32
6.0        11
7.0         7
8.0         4
9.0         4
11.0        2
12.0        1
13.0        1
10.0        1
Name: count, dtype: int64